In [1]:
import os
import sys
sys.path.append("C:/Users/yuuta/Documents/fashion")

In [10]:
import glob
import random
from utils.util import is_target_category, open_json
from utils.util import get_progress_percent

random.seed(42)

In [3]:
dirpath = glob.glob('C:/Users/yuuta/Documents/fashion/data/train/**/')
random.shuffle(dirpath)

In [9]:
import importlib
import utils.util
import pandas as pd
importlib.reload(utils.util)

<module 'utils.util' from 'C:\\Users/yuuta/Documents/fashion\\utils\\util.py'>

In [4]:
print(len(dirpath))

2140


In [6]:
# N-pair sampling
# triplet loss
# csvには，(anchor, positive, negative)で保存 (itemIdごとに)
N = 10

data = []

In [11]:
for cnt, user_dir in enumerate(dirpath):
    print(get_progress_percent(cnt, len(dirpath)), end='')
    coordinates = []
    _, user_id = os.path.split(os.path.split(user_dir)[0])
    user_coordinates_paths = glob.glob(user_dir + '*_new.json')
    for coordinate_path in user_coordinates_paths:
        coordinate = open_json(coordinate_path)
        # targetのもののみに絞る
        items = list(filter(is_target_category, coordinate["items"]))
        # 絞ったアイテムたちのidを取得
        item_ids = list(map(lambda x: x["itemId"], items))
        coordinates.append(item_ids)
    # N-pair samplingをする

    # 1. anchorのレイヤを決める
    for positive_layer_index, positive_coordinate in enumerate(coordinates):
        positive_set = set(positive_coordinate)
        # 2. anchorを決める
        for anchor_index, anchor_item in enumerate(positive_coordinate):
            # 3. そのレイヤはすべてpositive
            for positive_item in positive_coordinate[anchor_index+1:]:
                # 4. それ以外はすべてnegative
                for negative_coordinate in coordinates[positive_layer_index+1:]:
                    # 5. negativeのアイテムを決める
                    for negative_item in negative_coordinate:
                        # anchorと同じコーディネートに含まれるアイテムはネガティブじゃない
                        if negative_item in positive_set:
                            continue
                        data.append([anchor_item, positive_item, negative_item])


【******************* 

In [12]:
print(len(data))

186228507


In [13]:
# DataFrameを作成
df = pd.DataFrame(data)

# CSVファイルに保存
df.to_csv('C:/Users/yuuta/Documents/fashion/model_learning/compatibility/data/triplet_user.csv', index=False)